### ccRead Optimization 

Testing the feasibility of ccRead optimization through vectorization and use of comprehensions. To run this notebook place it in the root directory.

In [2]:
import random # for test sampling



import numpy as np
from numpy.lib.stride_tricks import as_strided
import numbers

import os

try:
    import plaidml.keras
    plaidml.keras.install_backend()

    from keras.models import load_model
    import keras.backend as K
except ImportError:
    try:
        from tensorflow.keras.models import load_model
        from tensorflow.keras import backend as K
    except ImportError:
        from keras.models import load_model
        import keras.backend as K
    finally:
        os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
        os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from PIL import Image, ImageCms
import cv2
import time
import os
import rawpy
from rawpy import LibRawNonFatalError, LibRawFatalError

position_model = load_model("libs/models/mlp_proposal.hdf5")
discriminator_model = load_model("libs/models/discriminator.hdf5")
high_precision_model = load_model("libs/models/highprecision_discriminator.hdf5")
#size_det_model = load_model("libs/models/size_model.hdf5")
#large_colorchip_regressor_model = load_model("libs/models/lcc_regressor.hdf5")
#size_model = load_model("libs/models/size_model.hdf5")

position_function = K.function(
    [position_model.layers[0].input, position_model.layers[1].input, K.learning_phase()],
    [position_model.layers[-1].output])

discriminator_function = K.function([discriminator_model.layers[0].input, K.learning_phase()],
                                         [discriminator_model.layers[-1].output])

high_precision_discriminator_function = K.function([high_precision_model.layers[10].input,
                                                         high_precision_model.layers[11].input,
                                                         high_precision_model.layers[0].input,
                                                         K.learning_phase()],
                                                        [high_precision_model.layers[-1].output])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


Set up base functions

In [5]:
def openImageFile(imgPath,
                  demosaic=rawpy.DemosaicAlgorithm.AHD):
    """ given an image path, attempts to return a numpy array image object
    """
    usr_gamma = 2.2
    gamma_value = (usr_gamma, usr_gamma)
    try:  # use rawpy to convert raw to openCV
        with rawpy.imread(imgPath) as raw:
            im = raw.postprocess(chromatic_aberration=(1, 1),
                                  demosaic_algorithm=demosaic,
                                  gamma=gamma_value,
                                  output_color=rawpy.ColorSpace.raw)

    # if it is not a raw format, just try and open it.
    except LibRawNonFatalError:
        bgr = cv2.imread(imgPath)
        im = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    except LibRawFatalError:
        raise
    return im

def scale_images_with_info(im, largest_dim=1875):
    """
    Function that scales images proportionally, and returns both the original image size as a tuple of image
    dimensions, and the scaled down image.
    :param im: Image to be scaled down.
    :type im: cv2 Image
    :param largest_dim: The largest dimension to be scaled down to.
    :type largest_dim: int
    :return: Returns both the original image dimensions as a tuple and the scaled image.
    :rtype: tuple, cv2 Image
    """
    image_height, image_width = im.shape[0:2]

    if image_width > image_height:
        reduced_im = cv2.resize(im, (largest_dim, round((largest_dim / image_width) * image_height)),
                                interpolation=cv2.INTER_AREA)
    else:
        reduced_im = cv2.resize(im, (round((largest_dim / image_height) * image_width), largest_dim),
                                interpolation=cv2.INTER_AREA)
    return (image_width, image_height), reduced_im

def _predict_uncertainty_position(x, n_iter=10):
    """
    Predicts with uncertainty the position of a color chip using the mean and variance.

    :param x: A list that contains both the RGB histogram and the HSV histogram of a given partition. Format of the
    list should be [rgb_histogram, hsv_histogram]
    :type x: list, ndarray
    :param n_iter: Number of iterations for the results. Most often would be the length of the list of histograms,
    though in a sorted list could be a value between 0 and length of list.
    :type n_iter: int
    :return: Returns the prediction and uncertainty. The prediction is a category exclusive probability.
    :rtype: list
    """
    result = []

    for i in range(n_iter):
        result.append(position_function([x[0], x[1], 1]))

    result = np.array(result)
    uncertainty = result.var(axis=0)
    prediction = result.mean(axis=0)
    return prediction, uncertainty

def _predict_uncertainty_discriminator(x, n_iter=10):
    """
    Predicts with uncertainty the probability that the given partition contains a color chip using the mean and
    variance.

    :param x: A list that contains both the RGB histogram and the HSV histogram of a given partition. Format of the
    list should be [rgb_histogram, hsv_histogram]
    :type x: list, ndarray
    :param n_iter: Number of iterations for the results. Most often would be the length of the list of histograms,
    though in a sorted list could be a value between 0 and length of list.
    :type n_iter: int
    :return: Returns the prediction and uncertainty. The prediction is a category exclusive probability.
    :rtype: list
    """
    result = []

    for i in range(n_iter):
        result.append(discriminator_function([x, 1]))

    result = np.array(result)
    uncertainty = result.var(axis=0)
    prediction = result.mean(axis=0)
    return prediction, uncertainty

def _predict_uncertainty_hp_discriminator(x, n_iter=10):
    """
    Predicts with uncertainty the probability that the given partition contains a color chip using the mean and
    variance. This uses the high precision model, which is more robust against false positives if

    Note: Currently, this model performs very poorly (relative to the normal discriminator model) against images
    with drastic white balance shift.

    :param x: A list that contains both the RGB histogram and the HSV histogram of a given partition. Format of the
    list should be [rgb_histogram, hsv_histogram]
    :type x: list, ndarray
    :param n_iter: Number of iterations for the results. Most often would be the length of the list of histograms,
    though in a sorted list could be a value between 0 and length of list.
    :type n_iter: int
    :return: Returns the prediction and uncertainty. The prediction is a category exclusive probability.
    :rtype: list
    """
    result = []

    for i in range(n_iter):
        result.append(high_precision_discriminator_function([x[0], x[1], x[2], 1]))

    result = np.array(result)
    uncertainty = result.var(axis=0)
    prediction = result.mean(axis=0)
    return prediction, uncertainty

def ocv_to_pil(im):
    """
    Converts an OCV image into PIL format. From
    https://stackoverflow.com/questions/43232813/convert-opencv-image-format-to-pil-image-format?noredirect=1&lq=1
    :param im: OpenCV image.
    :type im:
    :return:
    """

    pil_image = np.array(im)
    pil_image = Image.fromarray(pil_image)
    return pil_image


def cv2_rgb_hist(im_rgb):
    """
    a helper function to give cv2's histogram calculation similar syntax and result as pil.histogram()
    """
    hists = [cv2.calcHist([im_rgb], [channel], None, [256], [0, 256]).astype(int) for channel in [0,1,2]]
    pil_style_hist = np.concatenate(hists, axis=None)

    return pil_style_hist
   
def cv2_hsv_hist(im_hsv):
    """
    a helper function to give cv2's histogram calculation similar syntax and result as pil.histogram()
    """
    hists = [cv2.calcHist([im_hsv], [channel], None, [256], [0, 256]).astype(int) for channel in [0,1,2]]
    pil_style_hist = np.concatenate(hists, axis=None)

    return pil_style_hist

@jit
def np_img_crop(im, crop_box ):
    """
    im = numpy array image
    crop_box = a 4-tuple defining the left, upper, right, and lower pixel coordinate (inclusive)
        e.g.: x1, y1, x2, y2
    a helper function to give numpy image cropping similar syntax to PIL cropping.
    Note: PIL is inclusive, if you want 46x46: use (0, 0, 46, 46).
    """
    x1, y1, x2, y2 = crop_box
    # no values should be < 0
    y1 = max([0, y1-1])
    x1 = max([0, x1-1])
    # no values should be > the resolution max
    h_max, w_max = im.shape[0:2]
    y2 = min([h_max, y2+1])
    x2 = min([w_max, x2+1])
    #  Numpy indexing is exclusive, solve that
    return im[y1:y2, x1:x2, :]

Code for testing

### Optimize the `predict_color_chip_whitevals`


In [75]:
def test_predict_color_chip_whitevals(color_chip_image):
    """
    Takes the white values within the cropped CC image and averages them in RGB. The whitest values in the image is
    determined in the L*a*b color space, wherein only lightness values higher than (max lightness value - 1) is
    considered
    
    :param color_chip_image: The cropped color chip image.
    :type color_chip_image: Image
    :return: Returns a list of the averaged whitest values
    :rtype: list
    """
    
    cci_array = np.array(color_chip_image)
    ccil_array = cv2.cvtColor(color_chip_image, cv2.COLOR_RGB2Lab)
    width, height = ccil_array.shape[0], ccil_array.shape[1]
    # id max_lightness in ccil (lab space)
    max_lightness = ccil_array[...,0].max()
    # index cci_array based on conditions met in ccil_array
    white_pixels_rgbvals = cci_array[ ccil_array[...,0]> max_lightness-1]
    # det 
    white_pixels_average = np.average(white_pixels_rgbvals, axis=0)

    return list(white_pixels_average)

def predict_color_chip_whitevals(color_chip_image):
    """
    Takes the white values within the cropped CC image and averages them in RGB. The whitest values in the image is
    determined in the L*a*b color space, wherein only lightness values higher than (max lightness value - 1) is
    considered

    Converting from RGB to LAB color space is not supported under PIL, but was done through a solution from:
    https://gist.github.com/mrkn/28f95f95731a5a24e553
    :param color_chip_image: The cropped color chip image.
    :type color_chip_image: Image
    :return: Returns a list of the averaged whitest values
    :rtype: list
    """
    cci_array = np.array(color_chip_image)

    srgb_profile = ImageCms.createProfile("sRGB")
    lab_profile = ImageCms.createProfile("LAB")
    rgb2lab_transform = ImageCms.buildTransformFromOpenProfiles(srgb_profile, lab_profile, "RGB", "LAB")
    color_chip_image_lab = ImageCms.applyTransform(color_chip_image, rgb2lab_transform)

    ccil_array = np.array(color_chip_image_lab)
    width, height = ccil_array.shape[0], ccil_array.shape[1]

    lightness_dict = {}
    for row in range(width):
        for column in range(height):
            lightness = ccil_array[row][column][0]
            lightness_dict[(row, column)] = lightness

    max_lightness = max(list(lightness_dict.values()))
    white_pixels_indices = []
    for idx, lightness in enumerate(list(lightness_dict.values())):
        if lightness > (max_lightness - 1):
            white_pixels_indices.append(list(lightness_dict.keys())[idx])

    white_pixels_rgbvals = []
    for index in white_pixels_indices:
        row, column = index[0], index[1]
        white_pixels_rgbvals.append(cci_array[row][column])

    white_pixels_nparray = np.array(white_pixels_rgbvals)
    white_pixels_average = np.average(white_pixels_nparray, axis=0)

    return list(white_pixels_average)


In [76]:
imPath = './exampleImages/various_images/color_chip.jpg'
im = openImageFile(imPath)
original_size, img = scale_images_with_info(im)
display(img.shape)
img = im.copy()

(1875, 1707, 3)

In [77]:
start = time.time()
print(test_predict_color_chip_whitevals(im))
print(time.time() - start)

[254.25, 210.25, 158.25]
0.006119966506958008


In [81]:
test_times = []
orig_times = []
for i in range(0, 101):
    start = time.time()
    test_predict_color_chip_whitevals(im)
    test_times.append(time.time() - start)

    img = ocv_to_pil(img)
    start = time.time()
    predict_color_chip_whitevals(img)
    orig_times.append(time.time() - start)

avg_test = np.mean(test_times)
avg_orig = np.mean(orig_times)
print(f'avg from test func: {avg_test}')
print(f'avg from orig func: {avg_orig}')

avg from test func: 0.0020266452638229523
avg from orig func: 0.7686132157202994


#### results
`test_predict_color_chip_whitevals` appears much improved. However, the slight differences in the values is worthy of consideration. Possibly related to pil conversions. It is also worth verifying the colorspace is RGB at this point.